# Organize Results from the Experiments

In [22]:
from pathlib import Path
import pandas as pd
import logging
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [23]:
# get current working directory using pathlib
cwd = Path.cwd()

# define the path to the output folder
output_analysis_path = cwd / Path("output") / Path("output_analysis")

datatype = "ML" # "ML" or "DL"

results_path = output_analysis_path / Path(datatype)

# Check if the path exists and if it is a directory 
if results_path.exists() and results_path.is_dir():
    logging.info(f"Path exists: {results_path}")
else:
    raise Exception(f"Path does not exist: {results_path}")


In [24]:
def merge_csvs(base_path: Path) -> None:
    """
    Merge all CSV files in subfolders of the base path into a single CSV file.
    :param base_path: 
    :return: 
    """
    # Convert the base path to a Path object
    base_path = Path(base_path)
    
    # Iterate through each subfolder in the base path
    for subfolder in base_path.iterdir():
        if subfolder.is_dir():
            # Create a list to store dataframes for each subfolder
            df_list = []
            
            # Iterate through each CSV file in the current subfolder
            for csv_file in subfolder.glob('*.csv'):
                # Read the CSV into a dataframe
                df = pd.read_csv(csv_file)
                
                # Add a header with the filename (without .csv extension)
                header = pd.DataFrame([[csv_file.stem]], columns=df.columns[:1])  # Use the first column for the header
                
                # Append header and dataframe to the list
                df_list.append(header)
                df_list.append(df)
            
            # Concatenate all dataframes in the list
            if df_list:
                combined_df = pd.concat(df_list, ignore_index=True)
                
                # Define the output file path
                output_file = subfolder / f'{subfolder.name}_combined.csv'
                
                # Check if the combined file already exists
                if not output_file.exists():
                    # Save the combined dataframe as a CSV in the same folder
                    combined_df.to_csv(output_file, index=False)
                    logging.info(f'Saved combined CSV: {output_file}')
                else:
                    logging.info(f'Skipped saving: {output_file} already exists')

In [25]:
# Merge the CSV files in the results_path
merge_csvs(results_path)